In [ ]:
import json
import deepdoctection as dd
import markdownify

In [ ]:
page = dd.load_page("deepdocdetect_output.json")

In [ ]:
scale_x = 819 / 2481 # png width / root level width
scale_y = 1060 / 3508

Categories:
- table
- title
- text
- cell

In [ ]:
annotations = {
    "tables": [],
    "titles": [],
    "texts": []
}

In [ ]:
for annotation in page.annotations:
    if str(annotation.category_name) == 'LayoutType.table':
        annotations["tables"].append(annotation)
    elif str(annotation.category_name) == 'LayoutType.title':
        annotations["titles"].append(annotation)
    elif str(annotation.category_name) == 'LayoutType.text':
        annotations["texts"].append(annotation)

In [ ]:
for key, list in annotations.items():
    print(key, len(list))

In [ ]:
len(page.annotations)

In [ ]:
table_example = page.annotations[0]

In [ ]:
title_example = page.annotations[1]

In [ ]:
text_example = page.annotations[2]

In [ ]:
cell_example = page.annotations[5]

In [ ]:
with open("deepdocdetect_output.json", "r") as f:
    content = json.load(f)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
example = annotations["tables"][0]

In [ ]:
def get_table_values(table):
    table_relationships = table.relationships[next(iter(table.relationships))]
    annotation_id_to_value = {}
    for idx, annotation in enumerate(page.get_annotation(annotation_ids=table_relationships)):
        if len(annotation.relationships) > 0:
            for cell_annotation in page.get_annotation(
                annotation_ids=annotation.relationships[next(iter(annotation.relationships))]
            ):
                value = cell_annotation.sub_categories[next(iter(cell_annotation.sub_categories))].value
                annotation_id_to_value[annotation.annotation_id] = value
    return annotation_id_to_value

In [ ]:
def get_text_values(text):
    text_relationships = text.relationships[next(iter(text.relationships))]
    annotation_id_to_value = {}
    for annotation in page.get_annotation(annotation_ids=text_relationships):
        value = annotation.sub_categories[next(iter(annotation.sub_categories))].value
        annotation_id_to_value[annotation.annotation_id] = value
    return annotation_id_to_value

In [ ]:
table_html_list = table_example.sub_categories[next(iter(table_example.sub_categories))].value
table_html = "\n".join(table_html_list)
for annotation_id, value in get_table_values(table_example).items():
    table_html = table_html.replace(annotation_id, value)
print(table_html)

In [ ]:
img = plt.imread("examples/invoice-1.png")
fig, ax = plt.subplots(1, figsize=(15, 15))

# Display the image
ax.imshow(img)

rows = [{
    'ulx': example.bounding_box.ulx * scale_x,
    'uly': example.bounding_box.uly * scale_y,
    'lrx': example.bounding_box.lrx * scale_x,
    'lry': example.bounding_box.lry* scale_y,
    "content": "wow"
}]
for box in rows:
    print(box['uly'], box['lry'])
    # rect = patches.Rectangle((box['position_x0'], box['position_y0']), box['position_x1']-box['position_x0'], box['position_y1']-box['position_y0'], linewidth=1, edgecolor='r', facecolor='none')
    rect = patches.Rectangle((box['ulx'], box['uly']), box['lrx']-box['ulx'], box['lry']-box['uly'], linewidth=1, edgecolor='r', facecolor='none')
    print(rect)
    ax.add_patch(rect)
    #ax.annotate(box['content'], (box['ulx'], box['uly']), fontsize=8, color='r')

# Show the image
plt.show()